In [1]:
import tkinter as tk
import re

import ipynb_importer
from publish_subscribe import Reader

importing Jupyter notebook from publish_subscribe.ipynb


In [2]:
class MainPanel:
    def __init__(self, reader, publisher):
        self.reader = reader
        self.publisher = publisher
        self.all_authors = ['李白', '杜甫', '王維', '白居易']
        self.cur_author = ""
    
    def start(self):
        """
        显示主界面
        """
        self.root = tk.Tk()
        self.root.geometry('400x400')
        self.root.title("古诗订阅")
        self.root.iconbitmap('cat.ico')
        self.label = tk.Label(self.root, text="我的订阅", font=(None,12)).pack(pady=20)
        self.add_button = tk.Button(self.root, text="新增订阅", font=(None,12), command=self.add_subscription).pack(pady=40)
        self.root.mainloop()
        
    def add_subscription(self):
        """
        用户点击"新增订阅"按钮，显示该界面
        """
        self.add_panel = tk.Tk()
        self.add_panel.geometry('400x300')
        self.add_panel.iconbitmap('cat.ico')
        present_author_label = tk.Label(self.add_panel, text=f"已订阅作者：{self.reader.authors}", font=(None,10)).pack(pady=20)
        label = tk.Label(self.add_panel, text="新增订阅:", font=(None, 10)).pack(pady=20)
        self.new_author_entry = tk.Entry(self.add_panel)
        self.new_author_entry.pack()
        confirm_button = tk.Button(self.add_panel, text="确认", font=(None, 10), command=self.check_new_author).pack(pady=20)
        self.hint_label = tk.Label(self.add_panel, text="", font=(None, 10))
        self.hint_label.pack()
    
    def check_new_author(self):
        """
        用户点击"确认"按钮后，检查输入是否合法
        
        """
        new_author = self.new_author_entry.get()
        if new_author not in self.all_authors:
            self.hint_label.config(text=f"目前仅支持订阅:{self.all_authors}")
        elif new_author in self.reader.authors:
            self.hint_label.config(text="该作者已订阅，请重新输入")
        else:
            tmp_tk = tk.Tk()
            tmp_tk.geometry("300x200")
            tk.Label(tmp_tk, text="订阅成功", font=(None, 20)).pack(pady=20)
            tk.Button(tmp_tk, text="确认", font=(None, 12), command=tmp_tk.destroy).pack(pady=20)
            self.reader.subscribeToPublisher(new_author, self.publisher)
            self.add_panel.destroy()
            self.update_main_panel()
            
    def update_main_panel(self):
        """
        用户新增订阅后，更新主界面
        """
        for widget in self.root.winfo_children():
            widget.destroy()
        self.label = tk.Label(self.root, text="我的订阅", font=(None,12)).pack(pady=20)
        for author in self.reader.authors:
            button = tk.Button(self.root, text=author,width=13, font=(None,12))
            button.bind("<Button-1>", lambda event, x=author:self.show_titles(x))
            button.pack(pady=10)
        self.add_button = tk.Button(self.root, text="新增订阅", font=(None,12), command=self.add_subscription).pack(pady=40)
        self.root.update()
        
    def handler(self, x):
        title = x.get(x.curselection())
        self.show_detail(title)
        
    def show_detail(self, title):
        author = self.poems[title][1]
        content = (f"{author}《{title}》",) + self.poems[title][2:]
        self.add_detail = tk.Tk()
        self.add_detail.geometry('450x300')
        self.add_detail.iconbitmap('cat.ico')
        self.add_detail.title(title)
        for i in content:
            tk.Label(self.add_detail,text=i,font=(None,12)).pack(pady=5)
    
    def check_new_title(self):
        """
        用户点击"确认"按钮后，检查输入是否合法
        
        """
        new_title = self.new_title_entry.get()
        if new_title not in self.poems:
            self.hint_label.config(text=f"查无此标题")
        else:
            self.show_detail(new_title)
            
    def show_titles(self, author):
        self.add_panel = tk.Tk()
        self.add_panel.geometry('400x300')
        self.add_panel.iconbitmap('cat.ico')
        self.add_panel.title(author)
        self.poems = {}
        ScrollBar = tk.Scrollbar(self.add_panel)
        ScrollBar.pack(side=tk.RIGHT, fill = tk.Y)
        ListBox = tk.Listbox(self.add_panel, width=65, height=20, font=(None,12), yscrollcommand=ScrollBar.set)
        
        #Search Bar
        self.new_title_entry = tk.Entry(self.add_panel)
        self.new_title_entry.pack()
        self.hint_label = tk.Label(self.add_panel, text="", font=(None, 10))
        self.hint_label.pack()
        confirm_button = tk.Button(self.add_panel, text="搜索", font=(None, 10), command=self.check_new_title).pack(pady=20)
        
        for i in self.reader.poems[author]:
            ListBox.insert(tk.END, i[0])
            self.poems[i[0]]=i
        ListBox.bind('<<ListboxSelect>>',lambda event,x=ListBox:self.handler(x))
        ListBox.pack(side=tk.LEFT, fill=tk.BOTH)
        self.add_panel.mainloop()
        